In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.linalg as la
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import seaborn as sns

1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normali distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset
* Find the eigenvectors and eigenvalues using SVD. Check that they are two procedure yields to same result
* What percent of the total variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.
* Redefine the data in the basis yielded by the PCA procedure
* Plot the data points in the original and the new coordiantes as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.


In [ ]:
N = 1000

# x1
mu1 = 0
sigma1 = 1
x1 = np.random.normal( mu1, sigma1, N )

# x2
mu2 = 0
sigma2 = 3 
x2 = x1 + np.random.normal( mu2, sigma2, N )
#sig3 = m.sqrt( sigma1**2 + sigma2**2 + 2*np.cov( x1, x2 )[0][1] )
#np.std(x2)

# x3
x3 = 2*x1 + x2

#dataset = np.zeros( (N, 3) )
#dataset[:,0] = x1
#dataset[:,1] = x2
#dataset[:,2] = x3
dataset = np.array( [x1, x2, x3] )

# Eigenvalues, eigenvectors
cov_matrix = np.cov(dataset)

if la.det(cov_matrix) != 0:
    l, v = np.linalg.eig(cov_matrix)
    
# SVD
U, spectrum, Vt = la.svd(dataset)
l_svd = spectrum**2/(N-1)
print('\nEigenvalue of covariance matrix:', l )
print( 'Eigenvalue computed with SVD:', l_svd )

# Variability
print('\nVariability: ', l_svd / np.sum(l_svd) )

# percentage
print( '\nPercentage: ', l_svd[0] / (l_svd[0] + l_svd[1]) )


In [ ]:
print("\nVariability retained:",( l_svd[0] + l_svd[1] ) / np.sum(l_svd) )


Apply the eigenvectors matrix in order to rotate the data:

In [ ]:
X_new = np.dot(dataset.T, U).T

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
ax1[0].scatter( dataset[0,:], dataset[1,:] )
ax1[1].scatter( dataset[0,:], dataset[2,:] )
ax1[2].scatter( dataset[1,:], dataset[2,:] )

ax2[0].scatter( X_new[0,:], X_new[1,:] )
ax2[1].scatter( X_new[0,:], X_new[2,:] )
ax2[2].scatter( X_new[1,:], X_new[2,:] )

In [ ]:
data = pd.DataFrame({'x1':x1, 'x2':x2,'x3':x3})
sns.pairplot(data)

In [ ]:
data = pd.DataFrame(X_new.T)
sns.pairplot(data)

## 2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

In [ ]:
N = 1000
noise1 = np.zeros((10,N))
noise2 = np.zeros((10,N))
noise3 = np.zeros((10,N))

for i in range(10):
    mean = 0
    std = sigma1/50
    noise1[i] = np.random.normal(mean,std,N)
    std = sigma2/50
    noise2[i] = np.random.normal(mean,std,N)
    std = np.std(x3)/50
    noise2[i] = np.random.normal(mean,std,N)

x1 = np.random.normal( 0, sigma1, 1000 ) + noise1.sum( axis=0 )
x2 = np.random.normal( 0, sigma2, 1000 ) + x1 + noise2.sum( axis=0 )

In [ ]:
dataset = np.array( [x1, x2, x3] )
cov = np.cov( dataset )
l, V = np.linalg.eig(cov)
print ( "Covariance: ",cov )

U, spectrum, Vt = np.linalg.svd(dataset)
l_svd = spectrum**2/(N-1)
V_svd = U

print ('\neigenvalues:' ,l_svd)
print ('\neigenvectors:' ,V_svd)

In [ ]:
variability = l_svd / np.sum(l_svd)

print('variability per principal component:', variability)

#percentage
print( '\nPercentage: ', l_svd[0] / (l_svd[0] + l_svd[1]) )

print("\nVariability retained:",( l_svd[0] + l_svd[1] ) / np.sum(l_svd) )

In [ ]:
X_new = np.dot(dataset.T, U).T

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
ax1[0].scatter( dataset[0,:], dataset[1,:] )
ax1[1].scatter( dataset[0,:], dataset[2,:] )
ax1[2].scatter( dataset[1,:], dataset[2,:] )

ax2[0].scatter( X_new[0,:], X_new[1,:] )
ax2[1].scatter( X_new[0,:], X_new[2,:] )
ax2[2].scatter( X_new[1,:], X_new[2,:] )

In [ ]:
df = pd.DataFrame( { 'x1':x1, 'x2':x2, 'x3':( 2*x1 + x2 + noise2.sum( axis=0 ) + noise1.sum( axis=0 ) ) } )
sns.pairplot(df)
sns.pairplot(pd.DataFrame(dataset.T))

3 \. **Looking at an oscillating spring** (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset